# Application ontology for metabolomics
<br/>
<br/>

````{panels_fairplus}
:identifier_text: FCB024
:identifier_link: 'https://w3id.org/faircookbook/FCB024'
:difficulty_level: 4
:recipe_type: hands_on
:reading_time_minutes: 30
:intended_audience: principal_investigator, data_manager, data_scientist
:maturity_level: 3
:maturity_indicator: 33
:has_executable_code: yeah
:recipe_name: Building an application ontology for metabolomics - MSIO
````



## Overview
`MSIO an Application Ontology for Metabolomics. We detail how we use OBO Foundry ROBOT tool for generating
the ontology from a set of seeds, which have been established by manual selection of the key classes to build a
mild level ontology for experiments. MSIO specific classes are added the MSIO namespace but
remain compatible with OBI for which MSIO can become a module.` <br>


## Obtaining and installing robot

The robot tool is available from: http://robot.obolibrary.org/ with detailed instructions available from the official pages.
One may obtain the [latest official version](https://github.com/ontodev/robot/releases) or the [latest successful build](https://build.obolibrary.io/job/ontodev/job/robot/job/master/lastSuccessfulBuild/artifact/bin/robot.jar) from the obofoundry project. This the latter option we go for in this notebook as we want to test a new function available from version 1.7, onwards.


In [1]:
import os

In [2]:
robot_dir="robot_jar"
if not os.path.exists(robot_dir):
    os.makedirs(robot_dir)
    
# Check if a newer versio exists a:

get_robot_curl="curl -L https://build.obolibrary.io/job/ontodev/job/robot/job/master/lastSuccessfulBuild/artifact/bin/robot.jar > ./robot_jar/robot.jar"
try:
    os.system(get_robot_curl)
except IOError as ioe_curl:
            print(ioe_curl)

In [3]:
resources=[
"curl -L http://purl.obolibrary.org/obo/duo.owl   > ./ontology-sources-test/duo.owl",
"curl -L http://purl.obolibrary.org/obo/stato.owl > ./ontology-sources-test/stato.owl",
"curl -L http://purl.obolibrary.org/obo/chmo.owl  > ./ontology-sources-test/chmo.owl",
"curl -L http://purl.obolibrary.org/obo/iao.owl   > ./ontology-sources-test/iao.owl",
"curl -L http://purl.obolibrary.org/obo/obi.owl   > ./ontology-sources-test/obi.owl",
"curl -L http://purl.obolibrary.org/obo/ms.owl    > ./ontology-sources-test/ms.owl",
"curl -L https://raw.githubusercontent.com/MSI-Metabolomics-Standards-Initiative/nmrcv/master/nmrCV-corrected-namespace.owl > ./ontology-sources-test/nmrcv.owl",
"curl -L http://purl.obolibrary.org/obo/chebi.owl > ./ontology-sources-test/chebi.owl",
"curl -L http://purl.obolibrary.org/obo/uo.owl    > ./ontology-sources-test/uo.owl",
"curl -L http://purl.obolibrary.org/obo/stato.owl > ./ontology-sources-test/stato.owl"          
]

In [4]:
cwd = os.getcwd()
print(cwd)

/Users/philippe/Documents/git/FAIRplus-org/the-fair-cookbook/docs/content/recipes/ontology-robot


## 1. Downloading Reference Ontologies & Key Resources:

In [5]:
ontoresource_dir="ontology-sources-test"
if not os.path.exists(ontoresource_dir):
    os.makedirs(ontoresource_dir)
else:
    for resource in resources:
        try:
            print(resource)
            os.system(resource)
        except IOError as ioe_curl:
            print(ioe_curl)

## 2. Module Extraction from seeds using the MIREOT approach:
Using a number of seed files, which contains the classes manually selected by the ontology developers,
we use the [MIREOT method](http://dx.doi.org/10.3233/AO-2011-0087). The relevant ontology file can be found in the GitHub repository [here](https://github.com/ISA-tools/MSIO/tree/master/ontofoxInputs).

In [6]:
onto_outputdir="ontology-msio-build-modules"

extractors=[
"java -jar ./robot_jar/robot.jar extract --method MIREOT --input ./ontology-sources-test/chebi.owl --upper-terms ./ontology-select-classes/chebi-upper-terms.txt --lower-terms ./ontology-select-classes/chebi-lower-terms.txt --intermediates none --imports exclude --copy-ontology-annotations true --output ./ontology-msio-build-modules/chebi_mireot_module.owl",
"java -jar ./robot_jar/robot.jar extract --method MIREOT --input ./ontology-sources-test/chmo.owl  --lower-terms ./ontology-select-classes/chmo-lower-terms.txt  --intermediates all --copy-ontology-annotations true --output ./ontology-msio-build-modules/chmo_mireot_module.owl",
"java -jar ./robot_jar/robot.jar extract --method MIREOT --input ./ontology-sources-test/iao.owl   --lower-terms ./ontology-select-classes/iao-lower-terms.txt   --intermediates all --copy-ontology-annotations true --output ./ontology-msio-build-modules/iao_mireot_robot_module.owl",
"java -jar ./robot_jar/robot.jar extract --method MIREOT --input ./ontology-sources-test/obi.owl   --lower-terms ./ontology-select-classes/obi-lower-terms.txt   --intermediates all --copy-ontology-annotations true --output ./ontology-msio-build-modules/obi_mireot_robot_module.owl",
"java -jar ./robot_jar/robot.jar extract --method MIREOT --input ./ontology-sources-test/ms.owl --lower-terms ./ontology-select-classes/psims-lower-terms.txt --intermediates minimal --copy-ontology-annotations true --output ./ontology-msio-build-modules/psims_mireot_robot_module.owl",
"java -jar ./robot_jar/robot.jar extract --method MIREOT --input ./ontology-sources-test/uo.owl    --lower-terms ./ontology-select-classes/uo-lower-terms.txt    --intermediates minimal --copy-ontology-annotations true --output ./ontology-msio-build-modules/uo_mireot_robot_module.owl",
"java -jar ./robot_jar/robot.jar extract --method MIREOT --input ./ontology-sources-test/duo.owl   --lower-terms ./ontology-select-classes/duo-lower-terms.txt   --intermediates all --copy-ontology-annotations true --output ./ontology-msio-build-modules/duo_mireot_robot_module.owl"
] 

if not os.path.exists(onto_outputdir):
    os.makedirs(onto_outputdir)
else:
    for extractor in extractors:
        try:
            print(extractor)
            os.system(extractor)
        except IOError as ioe:
            print(ioe)    

## 3. Merging Extracted Modules into Application Ontology:
This step simply fuses all the input OWL files into one single output, avoiding use of import statements.<br/>
:octocat: A `merge` can be *undone* with `unmerge` command also available from `robot tool`.

In [7]:
if not os.path.exists("ontology-msio-build-results"):
    os.makedirs("ontology-msio-build-results")
    try:
        print(os.getcwd())
        os.system("java -jar ./robot_jar/robot.jar merge --input ./ontology-msio-build-modules/iao_mireot_robot_module.owl --input ./ontology-msio-build-modules/obi_mireot_robot_module.owl --input ./ontology-msio-build-modules/duo_mireot_robot_module.owl --input ./ontology-msio-build-modules/chmo_mireot_module.owl --input ./ontology-msio-build-modules/uo_mireot_robot_module.owl --input ./ontology-msio-build-modules/psims_mireot_robot_module.owl  --input ./ontology-msio-build-modules/chebi_mireot_module.owl --output ./ontology-msio-build-results/msio-test-merge.owl")
    except IOError as ioe_merge:
        print(ioe_merge)
else:
    try:
        print(os.getcwd())
        os.system("java -jar ./robot_jar/robot.jar merge --input ./ontology-msio-build-modules/iao_mireot_robot_module.owl --input ./ontology-msio-build-modules/obi_mireot_robot_module.owl --input ./ontology-msio-build-modules/duo_mireot_robot_module.owl --input ./ontology-msio-build-modules/chmo_mireot_module.owl --input ./ontology-msio-build-modules/uo_mireot_robot_module.owl --input ./ontology-msio-build-modules/psims_mireot_robot_module.owl  --input ./ontology-msio-build-modules/chebi_mireot_module.owl --output ./ontology-msio-build-results/msio-test-merge.owl")
    except IOError as ioe_merge:
        print(ioe_merge)    

/Users/philippe/Documents/git/FAIRplus-org/the-fair-cookbook/docs/content/recipes/ontology-robot


## 4. Working with ELK reasoner with Robot Materialize:
With this step, the goal is to ensure that the ontology is logically consistent. This can only be checked by running a `reasoner`, which will formally assess the various axioms used in the ontology and classify the classes bases on these constraints. Robot allows this step to be performed and provides various options as to which reasoner to offer. Here, the [ELK reasoner](https://www.cs.ox.ac.uk/isg/tools/ELK/) is used.

In [8]:
if not os.path.exists("ontology-msio-materialize"):
    os.makedirs("ontology-msio-materialize")
else:    
    try:
        os.system("java -jar ./robot_jar/robot.jar materialize --reasoner ELK  --input ./ontology-msio-build-results/msio-test-merge.owl  reduce --output ./ontology-msio-materialize/msio-test-materialize.owl")
    except IOError as ioe_reason:
        print(ioe_reason)    

## 5. Convert to OBO Format:
The module extraction, merging and reasoning function produce Ontology Web Language ([OWL](https://www.w3.org/TR/owl-features/)) documents. For reaching a wider audience and to allow use by some data entry tools, it is interesting to release the ontology in the [OBO format](https://owlcollab.github.io/oboformat/doc/GO.format.obo-1_4.html). The following section details how to invoke the command.

In [9]:
if not os.path.exists("ontology-msio-convert"):
    os.makedirs("ontology-msio-convert")
    try:
        os.system("java -jar ./robot_jar/robot.jar convert --input ./ontology-msio-materialize/msio-test-materialize.owl --format obo --output ./ontology-msio-convert/msio-test-conversion.obo")
    except IOError as ioe_reason:
        print(ioe_reason) 
else:    
    try:
        os.system("java -jar ./robot_jar/robot.jar convert --input ./ontology-msio-materialize/msio-test-materialize.owl --format obo --output ./ontology-msio-convert/msio-test-conversion.obo")
    except IOError as ioe_reason:
        print(ioe_reason)         

## 6. Importing stato and nmrCV ontology

In [10]:
# if not os.path.exists("ontology-msio-merge"):
#     os.makedirs("ontology-msio-merge")
# else: 
#     try:
#         os.system("java -jar robot.jar merge --input ./ontology-msio-materialize/msio-test-materialize.owl --input ./ontology-sources-modules/stato.owl --input ./ontology-sources-modules/nmrcv.owl --output ./ontology-msio-merge/msio-test-merge-round2.owl")
#     except IOError as ioe_convert:
#         print(ioe_convert)     

## 7. Export as Tabular Representation in Excel format
A new function in Robot tool (starting with version 1.7) provides the option of exporting the classes and properties of an ontology in a tabular representation, which can be useful to engage with users and curators for review and extension. The following section details how to invoke the command.

In [11]:
if not os.path.exists("ontology-msio-export"):
    os.makedirs("ontology-msio-export")
    try:
        os.system("java -jar ./robot_jar/robot.jar export --input ./ontology-msio-materialize/msio-test-materialize.owl --header 'IRI|ID|LABEL|SubClass Of|definition' --format xlsx --export ./ontology-msio-export/msio-test-export.xlsx")
    except IOError as ioe_convert:
        print(ioe_convert)
else: 
    try:
        os.system("java -jar ./robot_jar/robot.jar export --input ./ontology-msio-materialize/msio-test-materialize.owl --header 'IRI|ID|LABEL|SubClass Of|definition' --format xlsx --export ./ontology-msio-export/msio-test-export.xlsx")
    except IOError as ioe_convert:
        print(ioe_convert)

This ontology can also be found on GitHub https://github.com/ISA-tools/MSIO.


## References
````{dropdown} **References**
1. Jackson R.C., Balhoff J.P., Douglass E.,Harris N.L., Mungall C.J. and Overton J.A. ROBOT: A tool for automating ontology workflows. BMC Bioinformatics, vol. 20, July 2019.[https://doi.org/10.1186/s12859-019-3002-3](https://doi.org/10.1186/s12859-019-3002-3)
2. Hastings J., Owen G., Dekker A., Ennis M., Kale N., Muthukrishnan V., Turner S., Swainston N., Mendes P., Steinbeck C.(2016). ChEBI in 2016: Improved services and an expanding collection of metabolites. Nucleic Acids Res.[http://dx.doi.org/10.1093/nar/gks1146](http://dx.doi.org/10.1093/nar/gks1146)
10. Haug K., Cochrane K., Nainala V.C., Williams M., Chang J., Jayaseelan K.N. and O’Donovan C. MetaboLights: a resource evolving in response to the needs of its scientific community. Nucleic Acids Research, gkz1019, [https://doi.org/10.1093/nar/gkz1019](https://doi.org/10.1093/nar/gkz1019)
3. IAO: [https://github.com/information-artifact-ontology/IAO/](https://github.com/information-artifact-ontology/IAO/)
4. The Ontology for Biomedical Investigations, PLoS One. 2016 Apr 29;11(4):e0154556. eCollection 2016. [https://doi.org/10.1371/journal.pone.0154556](https://doi.org/10.1371/journal.pone.0154556). 
5. STATO: [https:stato-ontology.org](https:stato-ontology.org)
6. nmrCV: [http://nmrml.org/cv/](http://nmrml.org/cv/)
7. CHMO: [https://github.com/rsc-ontologies/rsc-cmo](https://github.com/rsc-ontologies/rsc-cmo)
8. UO: [https://github.com/bio-ontology-research-group/unit-ontology](https://github.com/bio-ontology-research-group/unit-ontology)
9. PSI-MS: [http://www.psidev.info/groups/controlled-vocabularies](http://www.psidev.info/groups/controlled-vocabularies)
````

## Authors
````{authors_fairplus}
Philippe: Writing - Original Draft
Susanna: Writing - Review & Editing, Funding Acquisition
Danielle: Writing - Review & Editing
````

## License
````{license_fairplus}
CC-BY-4.0
````